In [4]:
import pandas as pd
import os
import regex as re
import math
import numpy as np

In [5]:
basic = ['participant', 'session_num', 'timestamp', 'confidence', 'success']

gaze = ['gaze_0_x', 'gaze_0_y', 'gaze_0_z', 'gaze_1_x', 'gaze_1_y', 'gaze_1_z', 'gaze_angle_x', 'gaze_angle_y', 'gaze_distance_x', 'gaze_distance_y', 'gaze_distance']

pose = ['pose_Tx', 'pose_Ty', 'pose_Tz', 'pose_Rx', 'pose_Ry', 'pose_Rz', 'pose_distance']

action_units = ['AU01_c', 'AU02_c', 'AU04_c', 'AU05_c', 'AU06_c', 'AU07_c', 'AU09_c', 'AU10_c', 'AU12_c', 'AU14_c', 'AU15_c', 'AU17_c', 'AU20_c', 'AU23_c', 'AU25_c', 'AU26_c', 'AU28_c', 'AU45_c']

hand_picked = basic + gaze + pose + action_units

In [6]:
for participant in [11]:
    print()
    print(participant)
    data_path = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Open Face/p'+str(participant)
    out = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Open Face/p'+str(participant)+'_openface.csv'    
    
    master = pd.DataFrame(columns=hand_picked)
    
    for subdir, dirs, files in os.walk(data_path):
        for file in files:
            if 'openface' in file and 'csv' in file:
                path_to_file = data_path + '/' + file
                print(file)

                # Get Session Number 
                sess_regex1 = 's\d*_'
                sess_regex2 = 's\d*-\d*_'

                res1 = re.findall(sess_regex1, file)
                res2 = re.findall(sess_regex2, file)

                if (res1):
                    session = res1[0]
                    session = float(session[1:-1])
                else:
                    session = res2[0]
                    session = session[1:-1]
                    session = session.replace('-', '.')
                    session = float(session)

                process = pd.read_csv(path_to_file)
                process = process.drop(columns=['frame'])

                rename = {}
                for c in process.columns:
                    rename[c] = c[1:]
                process = process.rename(index=str, columns=rename)  

                process['participant'] = participant
                process['session_num'] = session

                # Construct Open Face Features 
                process['gaze_distance_x'] = 0.0
                process['gaze_distance_y'] = 0.0
                process['gaze_distance'] = 0.0
                process['pose_distance'] = 0.0

                process['gaze_angle_x'] = pd.to_numeric(process['gaze_angle_x'], errors='coerce')
                process['gaze_angle_y'] = pd.to_numeric(process['gaze_angle_y'], errors='coerce')            
                process['pose_Tx'] = pd.to_numeric(process['pose_Tx'], errors='coerce')
                process['pose_Ty'] = pd.to_numeric(process['pose_Ty'], errors='coerce')
                process['pose_Tz'] = pd.to_numeric(process['pose_Tz'], errors='coerce')

                for i, r in process.iterrows():    
                    if (r['success'] == 0):
                        process.at[i, 'gaze_distance_x'] = np.nan
                        process.at[i, 'gaze_distance_y'] = np.nan
                        process.at[i, 'gaze_distance'] = np.nan
                        process.at[i, 'pose_distance'] = np.nan
                    else:
                        d_x = (math.tan(r['gaze_angle_x'])*r['pose_Tz']) + r['pose_Tx']
                        d_y = (math.tan(r['gaze_angle_y'])*r['pose_Tz']) + r['pose_Ty']

                        process.at[i, 'gaze_distance_x'] = d_x
                        process.at[i, 'gaze_distance_y'] = d_y
                        process.at[i, 'gaze_distance'] = math.sqrt((d_y*d_y)+(d_x*d_x))
                        process.at[i, 'pose_distance'] = math.sqrt(math.pow(r['pose_Tx'], 2) + math.pow(r['pose_Ty'], 2) + math.pow(r['pose_Tz'], 2))            

                process = process[hand_picked]

                master = pd.concat([master,process], sort=True)
                
    master = master.sort_values(by=['session_num', 'timestamp'])
    master.to_csv(out, index=False)


11
p11_s22-2_openface.csv
p11_s22-1_openface.csv
p11_s20_openface.csv
p11_s19_openface.csv
p11_s18_openface.csv
p11_s17_openface.csv
p11_s16-2_openface.csv
p11_s16-1_openface.csv
p11_s15_openface.csv
p11_s14_openface.csv
p11_s13_openface.csv
p11_s12_openface.csv
p11_s11_openface.csv
p11_s10_openface.csv
p11_s9_openface.csv
p11_s8_openface.csv
p11_s7_openface.csv
p11_s6_openface.csv
p11_s5_openface.csv
p11_s4_openface.csv
p11_s3_openface.csv
p11_s2_openface.csv
p11_s23_openface.csv
